In [32]:
docs =['동물원 코끼리',
      '동물원 원숭이 바나나',
      '엄마 코끼리 아기 코끼리',
      '원숭이 바나나 코끼리 바나나']

In [33]:
doc_ls=[]
for doc in docs:
    doc_ls.append(doc.split(' '))
doc_ls    

[['동물원', '코끼리'],
 ['동물원', '원숭이', '바나나'],
 ['엄마', '코끼리', '아기', '코끼리'],
 ['원숭이', '바나나', '코끼리', '바나나']]

In [34]:
from collections import defaultdict
word2id = defaultdict(lambda:len(word2id))

#중복이 되는 토큰은 한번씩만 들어가게 된다.
for doc in doc_ls:
    for token in doc:
        word2id[token]  #토큰을 인덱스로

In [35]:
import numpy as np

TDM=np.zeros((len(word2id),len(doc_ls)),dtype=int)
print(TDM)
for i, doc in enumerate(doc_ls):
    for token in doc:
        TDM[word2id[token],i] +=1 #해당토큰위치(column)

#행렬표기(BOW와 차이점 : BOW는 1차원 배열)
TDM

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


array([[1, 1, 0, 0],
       [1, 0, 2, 1],
       [0, 1, 0, 1],
       [0, 1, 0, 2],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])

In [24]:
import pandas as pd
doc_names=['문서'+str(i) for i in range(len(doc_ls))]
print('doc_names',doc_names)
sorted_vocab=sorted((value,key) for key,value in word2id.items())
vocab=[v[1] for v in sorted_vocab]
df_TDM=pd.DataFrame(TDM,columns=doc_names)
df_TDM['단어']=vocab
df_TDM.set_index('단어')

doc_names ['문서0', '문서1', '문서2', '문서3']


,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
코끼리,1,0,2,1
원숭이,0,1,0,1
바나나,0,1,0,2
엄마,0,0,1,0
아기,0,0,1,0


In [ ]:
#SKLEARN

In [25]:
docs =['동물원 코끼리',
      '동물원 원숭이 바나나',
      '엄마 코끼리 아기 코끼리',
      '원숭이 바나나 코끼리 바나나']

In [26]:
#토큰 빈도계산 : CounterVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#선언
count_vect=CountVectorizer()
DTM=count_vect.fit_transform(docs)
DTM.toarray()

array([[1, 0, 0, 0, 0, 1],
       [1, 1, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 2],
       [0, 2, 0, 0, 1, 1]], dtype=int64)

In [27]:
DTM.toarray().T

array([[1, 1, 0, 0],
       [0, 1, 0, 2],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 1],
       [1, 0, 2, 1]], dtype=int64)

In [28]:
import pandas as pd

doc_names=['문서'+str(i) for i in range(len(doc_ls))]
vocab=count_vect.get_feature_names()
print(vocab)

df_TDM=pd.DataFrame(DTM.toarray().T,columns=doc_names)
df_TDM['단어']=vocab
df_TDM.set_index('단어')

['동물원', '바나나', '아기', '엄마', '원숭이', '코끼리']


,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
바나나,0,1,0,2
아기,0,0,1,0
엄마,0,0,1,0
원숭이,0,1,0,1
코끼리,1,0,2,1


In [ ]:
#GENSIM

In [29]:
docs =['동물원 코끼리',
      '동물원 원숭이 바나나',
      '엄마 코끼리 아기 코끼리',
      '원숭이 바나나 코끼리 바나나']

In [30]:
from gensim import corpora

#토큰화시키기
doc_ls=[]
for doc in docs:
    doc_ls.append(doc.split())
print(doc_ls)    
id2word=corpora.Dictionary(doc_ls)

TDM=[id2word.doc2bow(doc) for doc in doc_ls]
TDM

[['동물원', '코끼리'], ['동물원', '원숭이', '바나나'], ['엄마', '코끼리', '아기', '코끼리'], ['원숭이', '바나나', '코끼리', '바나나']]


[[(0, 1), (1, 1)],
 [(0, 1), (2, 1), (3, 1)],
 [(1, 2), (4, 1), (5, 1)],
 [(1, 1), (2, 2), (3, 1)]]

In [31]:
from gensim.matutils import sparse2full
import pandas as pd
import numpy as np


doc_names=['문서'+str(i) for i in range(len(doc_ls))]
vocab=[id2word[i] for i in id2word.keys()]
DTM_matrix=[sparse2full(doc,len(vocab)).tolist() for doc in TDM]

df_TDM=pd.DataFrame(np.array(DTM_matrix,dtype=int).T,columns=doc_names)
df_TDM['단어']=vocab
df_TDM.set_index('단어')

,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
코끼리,1,0,2,1
바나나,0,1,0,2
원숭이,0,1,0,1
아기,0,0,1,0
엄마,0,0,1,0
